In [1]:
from lagrangian import LagrangianSolver
from dataclasses import dataclass

import torch
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [2]:
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'

plt.rc('font', family='serif', serif="cmr10", size=18)
plt.rc('mathtext', fontset='cm', rm='serif')
plt.rc('axes', unicode_minus=False)

plt.rcParams['axes.formatter.use_mathtext'] = True

# Roller Coaster

We have $n$ carts on a track. Where some track is defined using a paramteric definition: $x(q_i)$ and $y(q_i)$, in here $q_i(t)$ is the generalized (constrained) coordinate on the track of the $i$-th cart.

To find the velocities in Euclidean space we have to transform from the generalized coordinates:
$$
\dot{x}_i = \frac{dx}{dq_i}\dot{q}_i;\qquad
\dot{y}_i = \frac{dy}{dq_i}\dot{q}_i
$$
We can easily numerically compute the derivates $x'$ and $y'$ using PyTorch's autograd system. For the potential we can simply use the $y$-coordinate.

This gives us the energy of the system:

$$
\begin{cases}
\displaystyle
T = \frac{m}{2} \sum_i (\dot{x}_i^2 + \dot{y}_i^2) \\ \\
\displaystyle
V = mg\sum_i y_i
\end{cases}
$$

And we constrain the carts to be always $R$ apart from each other:

$$
g_{i,{i+1}} = (x_i - x_{i+1})^2 + (y_i - y_{i+1})^2 - R^2 = 0
$$

Since we have $n$ carts, this means we will have $n-1$ of such links.

Now the tricky part comes with building up the track, first we have a straight part that transitions into a looping. When using a piecewise definition, the transition of going from straight to looping will cause numerical instabilities which case the constraint functions to drift. 

To fix this, I came up with a smooth transition function using a sigmoid function (as smoothed Heaviside) to ensure continuity of the track. It smoothly transitions the linear part into the looping.

You might use a spline definition of the track instead, but then you have to make sure you're able to compute the gradient w.r.t. generalized coordinate $q_i$.

In [3]:
@dataclass
class RollerCoaster(LagrangianSolver):
    r0: float
    m: float        # [kg] mass
    g: float=9.81   # [m/s]

    @staticmethod
    def mix(q): # logistic curve / sigmoid
        return 1-1/(1 + (5*(q + torch.pi/2)).exp())

    @staticmethod
    def x(q):
        flat = 3 * 10 * (torch.pi/2 + q)
        loop = 10 * (torch.pi/2 + q + 1.8*torch.cos(q))

        fac = RollerCoaster.mix(q)
        return (1-fac)*flat + fac*loop

    @staticmethod
    def y(q):
        flat = torch.zeros_like(q)
        loop = 10 * (torch.sin(q) + 1)
        
        fac = RollerCoaster.mix(q)
        return (1-fac)*flat + fac*loop
    
    def T(self, u: torch.Tensor) -> torch.Tensor:
        (q, qdot) = u[1:].view(2, self.n)

        dxdq = torch.autograd.functional.jacobian(self.x, q).diag()
        dydq = torch.autograd.functional.jacobian(self.y, q).diag()

        return self.m * ((dxdq**2 + dydq**2) * qdot.square()).sum() / 2
    
    def V(self, u: torch.Tensor) -> torch.Tensor:
        (s, sdot) = u[1:].view(2, self.n)
        return self.m * self.g * self.y(s).sum()
    
    def constraints(self, u: torch.Tensor) -> torch.Tensor:
        (s, sdot) = u[1:].view(2, self.n)
        r2 = self.x(s).diff().square() + self.y(s).diff().square()
        return (r2 - self.r0**2)

In [4]:
T = 5
N = 50*T
t = torch.linspace(0,T,N)

u0 = torch.tensor([
    -torch.pi + 0.4, -torch.pi + 0.3, -torch.pi + 0.2, -torch.pi + 0.1, -torch.pi,
    2., 2., 2., 2., 2.
], dtype=torch.float64)

solver = RollerCoaster(m=200, r0=RollerCoaster.x(u0[0]) - RollerCoaster.x(u0[1]))
soln = solver.solve(u0, t)
u = soln['u']

100%|██████████| 249/249 [00:21<00:00, 11.65it/s]


In [6]:
s = u.T[:u0.size(0)//2].T

fig, ax = plt.subplots(figsize=(10.8, 5.8))

s_track = torch.linspace(-torch.pi, 2*torch.pi, 100)

ax.plot(RollerCoaster.x(s_track), RollerCoaster.y(s_track), 'k--', lw=4)
carts, = ax.plot([], [], 'C0o-', ms=16)
ax.axis('equal')
ax.set_xlim(-20, 80)

ax.set_xlabel('$x$ / m')
ax.set_ylabel('$y$ / m')

fig.tight_layout()

def animate(i):
    idx = min(i, len(t))

    carts.set_data(RollerCoaster.x(s[idx]), RollerCoaster.y(s[idx]))

    return (carts,)

ani = animation.FuncAnimation(fig, animate, frames=len(t), blit=True)


plt.close()
ani.save('./assets/roller-coaster.mp4', writer="ffmpeg", fps=60, dpi=100)
# HTML(ani.to_jshtml())

Ignoring fixed x limits to fulfill fixed data aspect with adjustable data limits.
